In [1]:
!pip install kmedoids
!pip install gdown
!pip install python-mnist
!pip install pulp
# !pip install scikit-learn==0.22.2 --upgrade
!pip install scikit-learn==0.22.2.post1
!pip install zoopt
!pip install pyckmeans
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
ERROR: Could not build wheels for scikit-learn, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyckmeans: filename=pyckmeans-0.9.4-

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
# import os
# # !fusermount -u drive
# # !google-drive-ocamlfuse drive
# os.chdir("gdrive/My Drive/CFC-Master/Fair-Clustering-Codebase")

In [4]:
# %cd gdrive/My Drive/CFC-Master/Fair-Clustering-Codebase/
%cd /content/gdrive/MyDrive/CFC-master/Fair-Clustering-Codebase
#%cd Fair-Clustering-Codebase

/content/gdrive/MyDrive/CFC-master/Fair-Clustering-Codebase


In [5]:
!ls

Consensus-Fair-Clustering  s_TOY.npy		 U_idx_Yale.npy        V_idx_TOY.npy   y_TOY.npy
fair_clustering		   U_idx_DIGITS.npy	 utils.py	       V_idx_Yale.npy
models.py		   U_idx_MNIST_USPS.npy  V_idx_DIGITS.npy      X_DIGITS.npy
__pycache__		   U_idx_Office-31.npy	 V_idx_MNIST_USPS.npy  X_TOY.npy
s_DIGITS.npy		   U_idx_TOY.npy	 V_idx_Office-31.npy   y_DIGITS.npy


In [6]:
import os

os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8' #:4096:8

In [7]:
import sys
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
import random
import kmedoids
from sklearn.decomposition import PCA
from zoopt import Dimension, ValueType, Objective, Parameter, Opt, ExpOpt
import seaborn as sns
import subprocess
import torch

import warnings
warnings.filterwarnings('ignore')

from fair_clustering.eval.functions import * #[TO-DO] Write base class and derive metrics from it, temporary eval code

from fair_clustering.dataset import ExtendedYaleB, Office31, MNISTUSPS
from fair_clustering.algorithm import FairSpectral, FairKCenter, FairletDecomposition, ScalableFairletDecomposition

import matplotlib.pyplot as plt

In [8]:
# Set parameters related to dataset and get dataset

name = 'CIFAR100' #Choose between Office-31, MNIST_USPS, Yale, or DIGITS

if name == 'Office-31':
  dataset = Office31(exclude_domain='amazon', use_feature=True)
  X, y, s = dataset.data
elif name == 'CIFAR100':
  dataset = Cifar100(exclude_domain='omnivore', use_feature=False)
  X, y, s = dataset.data
elif name == 'MNIST_USPS':
  dataset = MNISTUSPS(download=True)
  X, y, s = dataset.data
elif name == 'Yale':
  dataset = ExtendedYaleB(download=True, resize=True)
  X, y, s = dataset.data
elif name == 'DIGITS':
  X, y, s = np.load('X_' + name + '.npy'), np.load('y_' + name + '.npy'), np.load('s_' + name + '.npy')

print(X.shape, y.shape, s.shape)


(3800, 256) (3800,) (3800,)


In [9]:
# Fairness Defense

In [10]:
from pyckmeans import CKmeans

In [11]:
# Remember we need the model class in the same directory to load our models so copy those over
!cp Consensus-Fair-Clustering/models.py ./
!cp Consensus-Fair-Clustering/utils.py ./
!ls

Consensus-Fair-Clustering  s_TOY.npy		 U_idx_Yale.npy        V_idx_TOY.npy   y_TOY.npy
fair_clustering		   U_idx_DIGITS.npy	 utils.py	       V_idx_Yale.npy
models.py		   U_idx_MNIST_USPS.npy  V_idx_DIGITS.npy      X_DIGITS.npy
__pycache__		   U_idx_Office-31.npy	 V_idx_MNIST_USPS.npy  X_TOY.npy
s_DIGITS.npy		   U_idx_TOY.npy	 V_idx_Office-31.npy   y_DIGITS.npy


In [12]:
from __future__ import division
from __future__ import print_function
import random
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

from models import GMLP, ClusteringLayer
from utils import get_A_r, sparse_mx_to_torch_sparse_tensor, target_distribution, aff

from scipy import sparse
from torch import nn

In [13]:
def Ncontrast(x_dis, adj_label, tau = 1):
    """
    compute the Ncontrast loss
    """
    x_dis = torch.exp( tau * x_dis)
    x_dis_sum = torch.sum(x_dis, 1)
    x_dis_sum_pos = torch.sum(x_dis*adj_label, 1)
    loss = -torch.log(x_dis_sum_pos * (x_dis_sum**(-1))+1e-8).mean()
    return loss

def get_batch(batch_size, idx_train, adj_label, features):
    """
    get a batch of feature & adjacency matrix
    """
    rand_indx = torch.tensor(np.random.choice(np.arange(adj_label.shape[0]), batch_size)).type(torch.long).cuda()
    rand_indx[0:len(idx_train)] = idx_train
    features_batch = features[rand_indx]
    adj_label_batch = adj_label[rand_indx,:][:,rand_indx]
    return features_batch, adj_label_batch

def train(model, CL, optimizer, s_idx0, s_idx1, bs, KL_div, tau, alpha, beta, idx_train, adj_label, features, Y, MSEL):
    features_batch, adj_label_batch = get_batch(bs, idx_train, adj_label, features)
    model.train()
    CL.train()

    optimizer.zero_grad()
    output, x_dis, embeddings = model(features_batch)

    output = CL(embeddings)
    output0, output1 = output[s_idx0], output[s_idx1]
    target0, target1 = target_distribution(output0).detach(), target_distribution(output1).detach()
    fair_loss = 0.5 * KL_div(output0.log(), target0) + 0.5 * KL_div(output1.log(), target1)

    loss_Ncontrast = Ncontrast(x_dis, adj_label_batch, tau = tau)

    predict0, predict1 = Y[s_idx0], Y[s_idx1]
    partition_loss = 0.5 * MSEL(aff(output0), aff(predict0)) + 0.5 * MSEL(aff(output1), aff(predict1))

    loss_train = alpha * fair_loss + loss_Ncontrast + beta * partition_loss

    loss_train.backward()
    optimizer.step()
    return


In [14]:
def ConsensusFairClusteringHelper(name, X_in, s_in, y_in, save, order=1, lr=0.01, weight_decay=5e-3, alpha=50.0, num_hidden=256, bs=3800, tau=2, epochs=3000, dropout=0.6):
  k = len(np.unique(y_in))

  if name == 'Office-31':
    beta = 100.0
    alpha = 1.0
    order = 1
  if name == 'MNIST_USPS':
    beta = 25.0
    alpha = 100.0
    order = 2
  if name == 'Yale':
    beta = 10.0
    alpha = 50.0
    order = 2
  if name == 'DIGITS':
    beta = 50.0
    alpha = 10.0
    order = 2
    num_hidden=36


  ckm = CKmeans(k=k, n_rep=100, p_samp=0.5, p_feat=0.5, random_state=42)
  ckm.fit(X_in)
  ckm_res = ckm.predict(X_in, return_cls=True)


  adj, features, labels = ckm_res.cmatrix, X_in, y_in
  adj = sparse.csr_matrix(adj)
  adj = sparse_mx_to_torch_sparse_tensor(adj).float()
  features = torch.FloatTensor(features).float()
  labels = torch.LongTensor(labels)
  idx_train = np.array(range(len(features)))
  idx_train = torch.LongTensor(idx_train)

  adj_label = get_A_r(adj, order)
  adj, adj_label, features, idx_train = adj.cuda(), adj_label.cuda(), features.cuda(), idx_train.cuda()

  s_idx0, s_idx1 = [], []
  for i in range(len(s_in)):
    if s_in[i] == 0:
      s_idx0.append(i)
    elif s_in[i] == 1:
      s_idx1.append(i)


  L = np.load('Consensus-Fair-Clustering/precomputed_labels/labels_' + name + '.npy')
  Y = np.zeros((len(s), k))
  for i,l in enumerate(L):
    Y[i,l] = 1.0
  Y = torch.FloatTensor(Y).float().cuda()
  MSEL = nn.MSELoss(reduction="sum")

  torch.manual_seed(42)
  torch.use_deterministic_algorithms(True)
  model = GMLP(nfeat=features.shape[1],
              nhid=num_hidden,
              nclass=labels.max().item() + 1,
              dropout=dropout,
              )

  torch.manual_seed(42)
  torch.use_deterministic_algorithms(True)
  CL = ClusteringLayer(cluster_number=k, hidden_dimension=num_hidden).cuda()

  optimizer = optim.Adam(model.get_parameters() + CL.get_parameters(), lr=lr, weight_decay=weight_decay)
  KL_div = nn.KLDivLoss(reduction="sum")
  model.cuda()
  features = features.cuda()
  labels = labels.cuda()
  idx_train = idx_train.cuda()

  for epoch in tqdm(range(epochs)):
    train(model, CL, optimizer, s_idx0, s_idx1, bs, KL_div, tau, alpha, beta, idx_train, adj_label, features, Y, MSEL)

  model.eval()
  logits, embeddings = model(features)
  CL.eval()
  preds = CL(embeddings)
  preds = preds.cpu().detach().numpy()
  pred_labels = np.argmax(preds, axis=1)

  return pred_labels


In [15]:
def ConsensusFairClustering(name, X_in, s_in, y_in, save):
  name_bal = {'Office-31': 0.5, 'MNIST_USPS': 0.3, 'DIGITS': 0.1, 'Yale': 0.1}
  while True: #Sometimes the model optimizes for a local minima which is why we can run enough times to get a good representation learnt
    cfc_labels = ConsensusFairClusteringHelper(name, X_in, s_in, y_in, save)
    if balance(cfc_labels, X_in, s_in) >= name_bal[name]: #threshold -> 0.5 for Office-31 and 0.3 (0.4) for MNIST_USPS and 0.1 for DIGITS and 0.1 for Yale
      break
  print("\nCompleted CFC model training.")
  return cfc_labels

In [16]:
# Trial run!
lbls = ConsensusFairClustering(name, X, s, y, save=False)
print(lbls)

100%|██████████| 3000/3000 [03:55<00:00, 12.76it/s]



Completed CFC model training.
[9 2 2 ... 2 2 9]


In [17]:
# Check to see metrics too!
print("balance: {}".format(balance(lbls, X, s)))
print("entropy: {}".format(entropy(lbls, s)))
print("nmi: {}".format(nmi(y, lbls)))
print("acc: {}".format(acc(y, lbls)))

balance: 0.3986013986013986
entropy: 2.8046766891714006
nmi: 0.2629514544169175
acc: 0.3821052631578947


In [18]:
''' the function that the attacker wants to optimize, computes the balance of the modified protected groups clustering'''
def attack_balance(solution):
  X_copy, s_copy = X.copy(), s.copy()
  flipped_labels = solution.get_x() #retrieve the solution from the optimization process and assigning it to the variable flipped_labels
  i = 0
  # for index in U_idx (all the indixes the attacker has access to)
  for idx in U_idx:
    # change the correct protected group of that sample with the one optimal for the attacker
    s_copy[idx] = flipped_labels[i]
    i += 1
  # compute the clustering again, but with the modified protected groups
  labels_sfd = ConsensusFairClustering(name, X_copy, s_copy, y, save=False)

  s_eval = []
  X_eval = []
  labels_sfd_eval = []
  # evaluate the solution given by the algo over the whole dataset, iterating through all the indices
  for idx in V_idx:
    s_eval.append(s_copy[idx])
    X_eval.append(X_copy[idx])
    labels_sfd_eval.append(labels_sfd[idx])
  s_eval = np.array(s_eval)
  X_eval = np.array(X_eval)
  labels_sfd_eval = np.array(labels_sfd_eval)
  #calculate the balance
  bal = balance(labels_sfd_eval, X_eval, s_eval)

  return bal


In [19]:
''' attacks the clustering algo and computes the metrics of to the attacked clustering '''

def process_solution(sol):
  X_copy, s_copy, y_copy = X.copy(), s.copy(), y.copy() # X is the data, y is the ground truth value and s is the protected group
  flipped_labels = sol.get_x() #retrieve the solution from the optimization process and assigning it to the variable flipped_labels
  i = 0
  # for index in U_idx (all the indixes the attacker has access to)
  for idx in U_idx:
    # change the correct protected group of that sample with the one optimal for the attacker
    s_copy[idx] = flipped_labels[i]
    i += 1

  # compute the clustering again, but with the modified protected groups
  labels_sfd = ConsensusFairClustering(name, X_copy, s_copy, y, save=False)

  s_eval = []
  X_eval = []
  labels_sfd_eval = []
  y_eval = []

  # evaluate the solution given by the algo over the whole dataset, iterating through all the indices
  for idx in V_idx:
    s_eval.append(s_copy[idx]) # append the modified protected groups
    X_eval.append(X_copy[idx]) # append the samples
    labels_sfd_eval.append(labels_sfd[idx]) # append the clusters
    y_eval.append(y_copy[idx]) # append the ground truth values

  # convert to numpy arrays
  s_eval = np.array(s_eval)
  X_eval = np.array(X_eval)
  labels_sfd_eval = np.array(labels_sfd_eval)
  y_eval = np.array(y_eval)

  # calculate the metrics and return them
  bal = balance(labels_sfd_eval, X_eval, s_eval)
  ent = entropy(labels_sfd_eval, s_eval)
  accuracy = acc(y_eval, labels_sfd_eval)
  nmi_score = nmi(y_eval, labels_sfd_eval)

  return (bal, ent, accuracy, nmi_score)

In [ ]:
# number of cluster is the number of all possible ground-truth values
n_clusters = len(np.unique(y))
print("# of clusters -> " + str(n_clusters))
n_trials = 1

U_idx_full, V_idx_full = np.load('U_idx_' + name + '.npy').tolist(), np.load('V_idx_' + name + '.npy').tolist()

cfc_pre_res = {
    0 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    1 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    2 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    3 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    4 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    5 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    6 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    7 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
}


cfc_post_res = {
    0 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    1 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    2 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    3 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    4 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    5 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    6 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    7 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
}

#U_idx is the list with all the indices of the samples, which might have the group membership switched
for percent, j in enumerate([int(0.125*len(U_idx_full)), int(0.25*len(U_idx_full)), int(0.375*len(U_idx_full)), int(0.5*len(U_idx_full)), int(0.625*len(U_idx_full)), int(0.75*len(U_idx_full)), int(0.875*len(U_idx_full)), int(len(U_idx_full))]):
  print('percent', percent)
  U_idx = U_idx_full[:j] #take the portion of U_idx
  V_idx = V_idx_full #indices of the full dataset

  for trial_idx in range(n_trials):
    #compute the clusters using CFC
    labels = ConsensusFairClustering(name, X, s, y, save=False) # X is the data, y is the ground truth value and s is the protected group
    
    s_test = []
    X_test = []
    labels_test = []
    y_test = []
    # for each index in all the indices of the dataset
    for idx in V_idx:
      s_test.append(s[idx]) # append the protected group of that sample 
      X_test.append(X[idx]) # append the data of that sample 
      labels_test.append(labels[idx]) # append the cluster assigned to that sample
      y_test.append(y[idx]) # append the real value of that sample

    # convert to numpy array
    s_test = np.array(s_test)
    X_test = np.array(X_test)
    labels_test = np.array(labels_test)
    y_test = np.array(y_test)

    # add to the pre-attack dictionary the calculated metrics (balance, entropy, accuracy and nmi)
    cfc_pre_res[percent]['BALANCE'].append(balance(labels_test, X_test, s_test))
    cfc_pre_res[percent]['ENTROPY'].append(entropy(labels_test, s_test))
    cfc_pre_res[percent]['ACC'].append(acc(y_test, labels_test))
    cfc_pre_res[percent]['NMI'].append(nmi(y_test, labels_test))

    # dim_size is the size of the indexes the attacker can modify at this point
    dim_size = len(U_idx)
    # Dimension is used to define the search space for optimization algorithms. dim_size indicated how many variables we want to optimize
    # the second parameter defines the range of values. Last parameter specifies that all variables have to be continuous.
    # So we are defining a search space for an optimization problem with dim_size dimensions, where each dimension allows continuous variables in the range [0, 1].
    dim = Dimension(dim_size, [[0, 1]]*dim_size, [False]*dim_size)
    #here we are optimizing the attack, with respect to the function attack_balance which is defined above and 'dim', which defines the search space for the optimization problem
    obj = Objective(attack_balance, dim)
    # we perform minimization optimization on obj (the optimization problem). The second parameter represents the maximum number of function 
    # evaluations or iterations that the optimization algorithm is allowed to perform ( 5 in this case )
    # the output stores the result of finding the values of the variables within the search space (dim) that minimize the 
    # objective function (attack_balance) and doing so within a maximum budget of 5 evaluations.
    solution = Opt.min(obj, Parameter(budget=5))
    
    # attack the algorithm and calculate the metrics with respect to the clustering post-attack
    pa_bal, pa_ent, pa_acc, pa_nmi = process_solution(solution)

    # store the post-attack metrics in the dictionary
    cfc_post_res[percent]['BALANCE'].append(pa_bal)
    cfc_post_res[percent]['ENTROPY'].append(pa_ent)
    cfc_post_res[percent]['ACC'].append(pa_acc)
    cfc_post_res[percent]['NMI'].append(pa_nmi)

# of clusters -> 10
percent 0


100%|██████████| 3000/3000 [03:51<00:00, 12.96it/s]



Completed CFC model training.


100%|██████████| 3000/3000 [02:18<00:00, 21.67it/s]



Completed CFC model training.


 82%|████████▏ | 2452/3000 [03:08<00:47, 11.58it/s]